Привет, меня зовут Артем Хуршудов. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Выводы и рассуждения получились содержательными, их было интересно читать.
* Радует, что тебе удалось разобраться с техникой бутстрапа.
* Отлично, что доверительный интервал и риск убытков посчитаны верно.
* Я оставил несколько советов, надеюсь, что они будут учтены в будущих проектах.
* Редко, кто сдает эту работу с первого раза, молодец!
* Проект зачтен. Удачи в будущих проектах!

## Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Содержание
* [Загрузка и подготовка данных](#open_file)
* [Обучение и проверка модели](#train_model)
* [Подготовка к расчёту прибыли](#calculation_preparation)
* [Расчёт прибыли и рисков](#calculate_risks)

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score, roc_auc_score, mean_squared_error, r2_score

from numpy.random import RandomState

RANDOM_STATE = RandomState(12345) 

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [2]:
class PredictionData(object):
    __slots__ = (        
        'features_train', 
        'target_train', 
        'features_valid', 
        'target_valid',
    )
    
    def __init__(
        self, features_train, target_train, 
        features_valid, target_valid, 
    ):
        self.features_train = features_train
        self.target_train = target_train 
        self.features_valid = features_valid
        self.target_valid = target_valid
        
    def __str__(self):
        return '\n'.join('размер {}: {}'.format(name, data.shape) for name, data in (
            ('features_train', self.features_train),
            ('target_train', self.target_train),
            ('features_valid', self.features_valid),
            ('target_valid', self.target_valid),
        ))
    
    def __iter__(self):
        return iter((
            self.features_train,
            self.target_train,
            self.features_valid,
            self.target_valid,
        ))
    
class ProfitAndVolume(object):
    __slots__ = (        
        'profit', 
        'volume', 
    )
    
    def __init__(self, profit, volume):
        self.profit = profit
        self.volume = volume 
        
    def __str__(self):
        return '\n'.join((
            'Прибыль (млн. руб): {:.2f}'.format(to_mil(self.profit)),
            'Суммарный объем сырья (тыс. баррелей): {:.2f}'.format(self.volume),
        ))
    
    def __iter__(self):
        return iter((self.profit, self.volume))

In [3]:
def analyze_df(df):
    rows_count, columns_count = df.shape
    print('Количество стобцов:', columns_count)
    print('Количество строк:', rows_count)
    display(df.head(10))
    display(df.describe())
    df.info()
    
def to_mil(num):
    return num / 1e6 

<a name="open_file"></a>
## Загрузка и подготовка данных

**Описание данных**:
- *id* — уникальный идентификатор скважины
- *f0*, *f1*, *f2* — три признака точек (неважно, что они означают, но сами признаки значимы);
- *product* — объём запасов в скважине (тыс. баррелей)

Загрузим данные из 3-х датасетов

In [4]:
geo_df_0 = pd.read_csv('/datasets/geo_data_0.csv') 
geo_df_1 = pd.read_csv('/datasets/geo_data_1.csv') 
geo_df_2 = pd.read_csv('/datasets/geo_data_2.csv')

dfs = (geo_df_0, geo_df_1, geo_df_2)

Проанализируем данные в них

In [5]:
analyze_df(geo_df_0)

Количество стобцов: 5
Количество строк: 100000


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
analyze_df(geo_df_1)

Количество стобцов: 5
Количество строк: 100000


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
analyze_df(geo_df_2)

Количество стобцов: 5
Количество строк: 100000


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Пропусков или иных аномалий не видно, посмотрим сколько в датасетах уникальных значений `id`

<div class="alert alert-block alert-success">
<b>Успех:</b> Загрузка и первичный анализ данных проведены успешно.
</div>

In [8]:
for idx, df in enumerate(dfs):
    print('Уникальные ли значения "id" в датасете {}: {}'.format(
        idx + 1, 
        df['id'].unique().shape[0] == df.shape[0],
    ))

Уникальные ли значения "id" в датасете 1: False
Уникальные ли значения "id" в датасете 2: False
Уникальные ли значения "id" в датасете 3: False


Видно, что значения не уникальны, поэтому удалим их 

In [9]:
for df in dfs:
    df.drop('id', axis=1, inplace=True)

<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец, что сделал эту проверку и удалил эту колонку. Абсолютно верное решение.
</div>

<a name="train_model"></a>
## Обучение и проверка модели

Напишем функцию, которая разобъет датасет на обучающую и валидационную выборку

In [10]:
def split_df(df):
    # Выделим фичи и целевой показатель product
    features = df.drop(columns=['product'])
    target = df['product']
    
    # Данные нужно разбить на 2 части: обучающую и валидационную, в соотношении 75:25
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=RANDOM_STATE)
    
    data = PredictionData(
        features_train,
        target_train,
        features_valid,
        target_valid, 
    )
    
    print(data)
        
    return data

Напишем функцию для обучения модели линейной регрессии и вычисления RMSE для каждого из датасетов.

In [11]:
def train_model(df):
    features_train, target_train, features_valid, target_valid = split_df(df)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = pd.Series(model.predict(features_valid), index=features_valid.index)
    
    print('RMSE: {:.2f}'.format(mean_squared_error(target_valid, predictions_valid) ** 0.5))
    print('Средний запас предсказанного сырья в регионе: {:.2f}'.format(predictions_valid.mean()))
    
    return target_valid, predictions_valid

Обучим модель на каждом из датасетов и сохраним предсказания и правильные ответы

In [12]:
target_valid_0, predictions_valid_0 = train_model(geo_df_0)

размер features_train: (75000, 3)
размер target_train: (75000,)
размер features_valid: (25000, 3)
размер target_valid: (25000,)
RMSE: 37.58
Средний запас предсказанного сырья в регионе: 92.59


In [13]:
target_valid_1, predictions_valid_1 = train_model(geo_df_1)

размер features_train: (75000, 3)
размер target_train: (75000,)
размер features_valid: (25000, 3)
размер target_valid: (25000,)
RMSE: 0.89
Средний запас предсказанного сырья в регионе: 68.77


In [14]:
target_valid_2, predictions_valid_2 = train_model(geo_df_2)

размер features_train: (75000, 3)
размер target_train: (75000,)
размер features_valid: (25000, 3)
размер target_valid: (25000,)
RMSE: 39.96
Средний запас предсказанного сырья в регионе: 95.09


**Выводы**:
* исходя из значений RMSE погрешность модели меньше всего для 2 региона
* среднее количество сырья примерно равно в 1 и 3 регионе (92.59 и 95.09 соотвественно), и больше в 1.4 раза, чем во 2

<div class="alert alert-block alert-success">
<b>Успех:</b> В целом, все сделано очень хорошо. Молодец, что напечатал размеры полученных наборов. Радует, что были использованы функции.
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Желательно проводить нормализацию признаков перед обучением линейной модели.
</div>

<a name="calculation_preparation"></a>
## Подготовка к расчёту прибыли

Сохраним все ключевые значения для расчетов в переменные

In [15]:
# количество исследуемых точек при разведке
RESEARCH_POINTS = 500 

# количество лучших исследуемых точек 
BEST_RESEARCH_POINTS = 200 

# бюджет на разработку скважин в регионе
BUDGET = 10 * 1e9 

# цена за 1000 баррелей сырья
PRODUCT_PRICE = 1000 * 450

# максимальная вероятность убытков
MAX_LOSS = 0.025

# количество выборок для bootstrap 
BOOTSTRAP_SAMPLES = 1000 

# доверительный интервал 
CONF_INTERVAL = 0.95

<div class="alert alert-block alert-success">
<b>Успех:</b> Радует, что в названиях константных переменных использованы только большие буквы.
</div>

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины, перед этим надо рассчитать бюджет на разработку одной скважины.

In [16]:
budget_per_point = BUDGET / BEST_RESEARCH_POINTS
min_product = budget_per_point / PRODUCT_PRICE
print('Минимальный объём сырья для безубыточной разработки новой скважины: {:.2f}'.format(min_product))

Минимальный объём сырья для безубыточной разработки новой скважины: 111.11


<div class="alert alert-block alert-success">
<b>Успех:</b> Точка безубыточности найдена верно.
</div>

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [17]:
def calculate_profit(target_valid, predictions_valid):
    research_points = predictions_valid.sample(RESEARCH_POINTS, replace=True, random_state=RANDOM_STATE)
    best_research_points = research_points.sort_values(ascending = False)[:BEST_RESEARCH_POINTS]
    
    product_volume = target_valid[best_research_points.index].sum()
    
    profit = PRODUCT_PRICE * product_volume - BUDGET
    
    return ProfitAndVolume(profit, product_volume)

<div class="alert alert-block alert-info">
<b>Совет: </b> В целом верно, но семплинг можно было сделать до функции.
</div>

Выполним ее для каждого из 3-х регионов

In [18]:
print(calculate_profit(target_valid_0, predictions_valid_0))

Прибыль (млн. руб): 334.51
Суммарный объем сырья (тыс. баррелей): 22965.57


In [19]:
print(calculate_profit(target_valid_1, predictions_valid_1))

Прибыль (млн. руб): 338.61
Суммарный объем сырья (тыс. баррелей): 22974.68


In [20]:
print(calculate_profit(target_valid_2, predictions_valid_2))

Прибыль (млн. руб): 407.97
Суммарный объем сырья (тыс. баррелей): 23128.83


**Выводы**: 
* получили, что минимальный объем сырья для безубыточной разработки новой скважины - 111.11 тыс. бареллей. Во всех 3-х регионах среднее предсказанное количество сырья меньше, следовательно не все скважины будут безубыточными.  
* взяли 500 точек из предсказаний, из которых отобрали 200 лучших и получили для них суммарный объем сырья, а из него вычислили прибыль. 

<div class="alert alert-block alert-success">
<b>Успех:</b> Полностью согласен с замечанием по поводу убыточности скважин.
</div>

<a name="calculate_risks"></a>
## Расчёт прибыли и рисков 

Применим технику Bootstrap, чтобы найти распределение прибыли.

In [21]:
def calculate_profit_bootstrap(target_valid, predictions_valid):
    values = []

    for i in range(BOOTSTRAP_SAMPLES): 
        profit_and_volume = calculate_profit(target_valid, predictions_valid)     
        values.append(profit_and_volume.profit)
            
    values = pd.Series(values)
    profit_mean = values.mean()

    conf_lower = values.quantile((1 - CONF_INTERVAL) / 2)
    conf_upper = values.quantile((1 + CONF_INTERVAL) / 2) 
    
    loss_probability = (values < 0).sum() / BOOTSTRAP_SAMPLES 
    
    print('Средняя прибыль (млн. руб): {:.2f}'.format(to_mil(profit_mean)))
    print('95% доверительный интервал (млн. руб): ({:.2f}, {:.2f})'.format(to_mil(conf_lower), to_mil(conf_upper)))

    if loss_probability < MAX_LOSS: 
        print('Вероятность убытков равна {}, что меньше чем максимальная допустимая вероятность {}'.format(loss_probability, MAX_LOSS))
    else:
        print('Вероятность убытков равна {}, что больше чем максимальная допустимая вероятность {}'.format(loss_probability, MAX_LOSS))    

    return profit_mean, conf_lower, conf_upper, loss_probability

In [22]:
profit_and_risks = []

In [23]:
profit_and_risks.append(calculate_profit_bootstrap(target_valid_0, predictions_valid_0))

Средняя прибыль (млн. руб): 394.14
95% доверительный интервал (млн. руб): (-69.45, 915.50)
Вероятность убытков равна 0.061, что больше чем максимальная допустимая вероятность 0.025


In [24]:
profit_and_risks.append(calculate_profit_bootstrap(target_valid_1, predictions_valid_1))

Средняя прибыль (млн. руб): 454.46
95% доверительный интервал (млн. руб): (64.53, 855.13)
Вероятность убытков равна 0.007, что меньше чем максимальная допустимая вероятность 0.025


In [25]:
profit_and_risks.append(calculate_profit_bootstrap(target_valid_2, predictions_valid_2))

Средняя прибыль (млн. руб): 353.74
95% доверительный интервал (млн. руб): (-162.65, 847.76)
Вероятность убытков равна 0.076, что больше чем максимальная допустимая вероятность 0.025


In [26]:
profit_and_risks_df = pd.DataFrame(profit_and_risks, columns = ['profit_mean', 'conf_lower', 'conf_upper', 'loss_probability'])
profit_and_risks_df.head(3)

,profit_mean,conf_lower,conf_upper,loss_probability
0,3.941415e+08,-6.945057e+07,9.154962e+08,0.061
1,4.544582e+08,6.452857e+07,8.551335e+08,0.007
2,3.537407e+08,-1.626509e+08,8.477618e+08,0.076


<div class="alert alert-block alert-success">
<b>Успех:</b> Все статистики посчитаны верно. Радует, что все регионы были собраны в одну таблицу, так мы можем намного проще их сравнить между собой.
</div>

**Выводы**:
* Из таблицы видно, что единственный регион, где вероятность убытков ниже, чем допустимое значение - **второй** (первый, если использовать индексацию с 0), поэтому для разработки скважин надо выбрать его.
* Также у второго региона самый узкий доверительный интервал и самая выскокая средняя прибыль.

<div class="alert alert-block alert-success">
<b>Успех:</b> Согласен с выбором региона. Молодец, что уточнил по поводу нумерации.
</div>

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован